In [ ]:
from pandas.core.interchange.dataframe_protocol import DataFrame
%reset -f

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matplotlib.gridspec as gridspec

from matplotlib.patches import ConnectionPatch
from MINE.Log import Log
from MINE.Analysis import SessionAnalytics, ExperimentAnalytics
from MINE.StreamFilter import IStreamFilter, TimestampStreamFilter
from MINE.SessionFilters import ISessionFilter, ContainsStreamSessionFilter, ContainsMarkersSessionFilter

mpl.rcParams['figure.dpi'] = 300

In [ ]:
def stream_filter() -> ContainsStreamSessionFilter:
    return ContainsStreamSessionFilter([
        "Marker",
        "Av Rating Continuous",
        "OcoSense_OCO_14640BA8E2DD [98BE107F-3760-7351-78D3-DFB1DE240AE5]",
        "HR"
    ])

def marker_filter() -> ContainsMarkersSessionFilter:
    return ContainsMarkersSessionFilter("Marker", [
        "Video: Be a floater, Event: VideoStart",
        "Video: Be a floater, Event: VideoEnd",
        "Video: RNLI Respect the water “Ladbible short film”, Event: VideoStart",
        "Video: RNLI Respect the water “Ladbible short film”, Event: VideoEnd",
        "Video: “Evans story”, Event: VideoStart",
        "Video: “Evans story”, Event: VideoEnd",
        "Video: “Little girl being rescued by RNLI”, Event: VideoStart",
        "Video: “Little girl being rescued by RNLI”, Event: VideoEnd",
        "Video: Alfie’s phone, Event: VideoStart",
        "Video: Alfie’s phone, Event: VideoEnd",
        "Video: “Float to Live”, Event: VideoStart",
        "Video: “Float to Live”, Event: VideoEnd",
        "Video: Respect the Water via the NWSF\u202f‘make the right call’, Event: VideoStart",
        "Video: Respect the Water via the NWSF\u202f‘make the right call’, Event: VideoEnd",
        "Video: “Seaside safety song”, Event: VideoStart",
        "Video: “Seaside safety song”, Event: VideoEnd",
        "Video: RNLI the breath test, Event: VideoStart",
        "Video: RNLI the breath test, Event: VideoEnd",
        "Video: RNLI Christmas bed-time story, Event: VideoStart",
        "Video: RNLI Christmas bed-time story, Event: VideoEnd",
        "Video: “RNLI: The heart-breaking story of Liam Hall”, Event: VideoStart",
        "Video: “RNLI: The heart-breaking story of Liam Hall”, Event: VideoEnd",
    ])

In [ ]:
def video_data() -> pd.DataFrame:
    return pd.DataFrame(columns=["Video_ID", "Video_Name"], data=[
        ["Video 1", "Video: Be a floater"],
        ["Video 2", "Video: RNLI Respect the water “Ladbible short film”"],
        ["Video 3", "Video: “Evans story”"],
        ["Video 4", "Video: “Little girl being rescued by RNLI”"],
        ["Video 5", "Video: Alfie’s phone"],
        ["Video 6", "Video: “Float to Live”"],
        ["Video 7", "Video: Respect the Water via the NWSF\u202f‘make the right call’"],
        ["Video 8", "Video: “Seaside safety song”"],
        ["Video 9", "Video: RNLI the breath test"],
        ["Video 10", "Video: RNLI Christmas bed-time story"],
        ["Video 11", "Video: “RNLI: The heart-breaking story of Liam Hall”"],
    ])

def participant_data() -> pd.DataFrame:
     return pd.DataFrame(columns=["Participant_ID", "File_Path"], data=[
         ["P01 - 535679", "V:/Data/Analysis/RNLI/participant 1.xdf"],
         ["P02 - 888462", "V:/Data/Analysis/RNLI/participant 2.xdf"],
         ["P03 - 499031", "V:/Data/Analysis/RNLI/participant 3.xdf"],
         ["P04.0 - 832362", "V:/Data/Analysis/RNLI/participant 4.0.xdf"],
         ["P04.1 - 832362", "V:/Data/Analysis/RNLI/participant 4.1.xdf"],
         ["P05", "V:/Data/Analysis/RNLI/participant 5.xdf"],
         ["P06", "V:/Data/Analysis/RNLI/participant 6.xdf"],
         ["P07", "V:/Data/Analysis/RNLI/participant 7.xdf"],
         ["P08", "V:/Data/Analysis/RNLI/participant 8.xdf"],
         ["P09", "V:/Data/Analysis/RNLI/participant 9.xdf"],
         ["P10 - 999149", "V:/Data/Analysis/RNLI/participant 10.xdf"],
         ["P11 - 153327", "V:/Data/Analysis/RNLI/participant 11.xdf"],
         ["P12", "V:/Data/Analysis/RNLI/participant 12.xdf"],
         ["P13", "V:/Data/Analysis/RNLI/participant 13.xdf"],
         ["P14", "V:/Data/Analysis/RNLI/participant 14.xdf"],
         ["P15 - 156103", "V:/Data/Analysis/RNLI/participant 15.xdf"],
         ["P16 - 701399", "V:/Data/Analysis/RNLI/participant 16.xdf"]
     ])

In [ ]:
_experiment_analytics: ExperimentAnalytics = ExperimentAnalytics.create_from_paths(participant_data())
_experiment_analytics: ExperimentAnalytics = _experiment_analytics.get_filtered_subset([
    marker_filter(),
    stream_filter()
])

In [ ]:
def add_stream_values(session_analytics: SessionAnalytics, videos: pd.DataFrame, video_name: str, stream_name: str, start: float, end: float, normalised_time: bool = False):
    timestamp_filter = TimestampStreamFilter(start, end)
    subset_analytics = session_analytics.get_filtered_subset(timestamp_filter)

    stream_subset = subset_analytics.stream_data_dictionary[stream_name]

    values = stream_subset["Value"].tolist()
    timestamps = stream_subset["Timestamp"].tolist()
    normalised_timestamps = timestamps - start

    if f"{stream_name}_Timestamps" not in videos: videos[f"{stream_name}_Timestamps"] = pd.Series(dtype=object)
    if f"{stream_name}_Values" not in videos: videos[f"{stream_name}_Values"] = pd.Series(dtype=object)

    row_index = videos.index[videos['Video_Name'] == video_name].to_list()[0]
    videos.at[row_index, f"{stream_name}_Timestamps"] = normalised_timestamps if normalised_time else timestamps
    videos.at[row_index, f"{stream_name}_Values"] = values

def populate_video_data(session_analytics: SessionAnalytics, videos: pd.DataFrame):
    marker_pairs = session_analytics.get_paired_markers("Marker", "VideoStart", "VideoEnd")

    # Add in the timestamp data to the dataframe
    for _, video_row in videos.iterrows():
        video_name = video_row.loc["Video_Name"]
        marker_row = marker_pairs[marker_pairs["Start Marker"].str.contains(video_name, regex=False)]

        start_time = marker_row["Start Timestamp"].iloc[0]
        end_time = marker_row["End Timestamp"].iloc[0]

        videos.loc[videos['Video_Name'] == video_name, 'Start_Timestamp'] = start_time
        videos.loc[videos['Video_Name'] == video_name, 'End_Timestamp'] = end_time

        add_stream_values(session_analytics, videos, video_name, "PPG_GRN", start_time, end_time, normalised_time=True)
        add_stream_values(session_analytics, videos, video_name, "PPG_RED", start_time, end_time, normalised_time=True)
        add_stream_values(session_analytics, videos, video_name, "PPG_IR", start_time, end_time, normalised_time=True)

In [ ]:
def get_maximum_timestamp(videos: pd.DataFrame) -> float:
    maximum_timestamp: float = 0
    for _, video_row in videos.iterrows():
        maximum_green_timestamp = max(video_row["PPG_GRN_Timestamps"])
        maximum_red_timestamp = max(video_row["PPG_RED_Timestamps"])
        maximum_infra_red_timestamp = max(video_row["PPG_IR_Timestamps"])

        maximum_timestamp = max(maximum_timestamp, maximum_green_timestamp, maximum_red_timestamp, maximum_infra_red_timestamp)

    return maximum_timestamp

In [ ]:
def get_range(values: np.ndarray, multiplier: float) -> list[float]:
    min_value = min(values)
    max_value = max(values)

    value_range = max_value - min_value
    mid_point = min_value + value_range / 2

    min_limit = mid_point - (value_range / 2) * multiplier
    max_limit = mid_point + (value_range / 2) * multiplier

    return [min_limit, max_limit]

def plot_ppg_axis(axis: plt.Axes, video_row: pd.Series, data_prefix: str, colour: str, x_limits: list[float], plot_index: list[int]):
    timestamps = video_row[f"{data_prefix}_Timestamps"]
    values = video_row[f"{data_prefix}_Values"]

    axis.plot(timestamps, values, c = colour)

    y_limits = get_range(values, 1.1)

    axis.set_autoscale_on(False)
    axis.set_xlim(x_limits[0], x_limits[1])
    axis.set_ylim(y_limits[0], y_limits[1])

    plot_index[0] += 1

def seperator(plot_index: list[int]):
    plot_index[0] += 1

In [ ]:
def plot_raw_ppg_data(session_id: str, videos: pd.DataFrame):
    figure_row_count = len(videos) * 3 + len(videos) - 1
    figure = plt.figure(figsize=(80, 30))
    figure.suptitle(f"Session: {session_id}", fontsize=16)
    figure_grid_space = gridspec.GridSpec(figure_row_count, 1)

    x_limits = [0, get_maximum_timestamp(videos)]

    #Plot the data
    plot_index: list[int] = [0]
    for video_index, video_row in videos.iterrows():
        video_id = video_row["Video_ID"]

        # Plot the green light stream
        green_axis = figure.add_subplot(figure_grid_space[plot_index[0]])
        plot_ppg_axis(green_axis, video_row, "PPG_GRN", "green", x_limits, plot_index)

        # Plot the red light stream
        red_axis = figure.add_subplot(figure_grid_space[plot_index[0]])
        plot_ppg_axis(red_axis, video_row, "PPG_RED", "orange", x_limits, plot_index)

        # Plot the orange light stream
        infra_red_axis = figure.add_subplot(figure_grid_space[plot_index[0]])
        plot_ppg_axis(infra_red_axis, video_row, "PPG_IR", "red", x_limits, plot_index)

        # Draw Grouping Info
        red_axis.text(-0.015, 0.5, video_id, transform=red_axis.transAxes, rotation = 90, ha='right', va='center', fontsize=12, color='black')
        #line = ConnectionPatch(xyA=(0.5, 1.0), coordsA='axes fraction', xyB=(0.5, 0.0), coordsB='axes fraction', axesA=green_axis, axesB=infra_red_stream, color='black',linewidth=2)
        #figure.add_artist(line)

        # Add the breaker
        seperator(plot_index)

    plt.subplots_adjust(left=0.05, right= 0.99, top=0.95, bottom=0.05, hspace=0.6)
    plt.savefig(f"V:/Exports/{session_id} PPG Data.png")
    plt.close()

    Log.message(f"Saved File: {session_id} PPG Data.png")

In [ ]:
def process():
    for _, row in _experiment_analytics.analytics_dataframe.iterrows():
        session_id = row["Session_ID"]
        session_analytics = row["Analysis_Object"]

        session_video_data = video_data()
        populate_video_data(session_analytics, session_video_data)
        plot_raw_ppg_data(session_id, session_video_data)

process()

In [ ]:
"""
timestamp_filter = TimestampStreamFilter(11528.00403109542, 11585.220626990775)
subset = stream_analytics.get_filtered_subset(timestamp_filter)

stream = stream_analytics.stream_data_dictionary["PPG_IR"]

_values = np.array(stream["Value"].tolist())
_timestamps = stream["Timestamp"].to_numpy()

plt.figure(figsize=(10, 5))
plt.plot(_timestamps, _values)

plt.savefig("V:/PPG_IR.png")

"[Video: “Little girl being rescued by RNLI”, Event: VideoStart]"
"""

In [ ]:
"""
for i, video in enumerate(videos):
    base_row = i * 4  # 3 plots + 1 spacer
    for j in range(3):
        ax = fig.add_subplot(gs[base_row + j])
        ax.plot(x, ppg_ir, label=plot_labels[j], color=colors[j])
        if j == 1:
            ax.text(-0.025, 0.5, plot_labels[j], transform=ax.transAxes, rotation = 90, ha='right', va='center', fontsize=12, color='black')
        ax.legend()

axes = fig.get_axes()
con = ConnectionPatch(xyA=(0.5, 1.0), coordsA='axes fraction', xyB=(0.5, 0.0), coordsB='axes fraction', axesA=axes[0], axesB=axes[1], color='black',linewidth=2)

fig.add_artist(con)


plt.subplots_adjust(left=0.05, right= 0.95, top=0.95, bottom=0.05, hspace=0.6)
plt.savefig('V:/test.png')
plt.show()
"""

In [ ]:
"""
# Simulated PPG data
x = np.linspace(0, 10, 100)
ppg_ir = np.sin(x)
ppg_grn = np.cos(x)
ppg_red = np.sin(x + np.pi/4)

videos = ['Video 1', 'Video 2', 'Video 3']
colors = ['PPG_IR', 'PPG_GRN', 'PPG_RED']
signals = [ppg_ir, ppg_grn, ppg_red]

fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(10, 8), sharex=True)
fig.suptitle('Person 1', fontsize=16)

for i, ax in enumerate(axes):
    ax.plot(x, ppg_ir, label='PPG_IR')
    ax.plot(x, ppg_grn, label='PPG_GRN')
    ax.plot(x, ppg_red, label='PPG_RED')
    ax.set_title(videos[i])
    ax.legend(loc='upper right')
    ax.set_ylabel('Amplitude')

axes[-1].set_xlabel('Time (s)')
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()
"""

In [ ]:
"""
videos = ['Video 1', 'Video 2', 'Video 3']

fig, axes = plt.subplots(len(videos) * 3, 1, figsize=(20, 15), sharex=True)
plt.suptitle(f'Person {i+1}', fontsize=16)

for i, video in enumerate(videos):
    subplot_index = i * 3
    axes[subplot_index].title.set_text(f'Video {i}')
    axes[subplot_index].plot(x, ppg_ir, label='PPG_GRN', c = 'green')
    axes[subplot_index + 1].plot(x, ppg_ir, label='PPG_RED', c = 'orange')
    axes[subplot_index + 2].plot(x, ppg_ir, label='PPG_IR', c = 'red')

for _, axis in enumerate(axes): axis.legend()

plt.tight_layout()
plt.show()
"""

In [ ]:
"""
from matplotlib.patches import ConnectionPatch
from matplotlib.patches import FancyArrowPatch
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import numpy as np

videos = ['Video 1', 'Video 2', 'Video 3']
x = np.linspace(0, 10, 100)
ppg_ir = np.sin(x)

fig = plt.figure(figsize=(20, 15))
gs = gridspec.GridSpec(11, 1, height_ratios=[1]*3 + [0.2] + [1]*3 + [0.2] + [1]*3)
fig.suptitle('Person 1', fontsize=16)

plot_labels = ['PPG_GRN', 'PPG_RED', 'PPG_IR']
colors = ['green', 'orange', 'red']

for i, video in enumerate(videos):
    base_row = i * 4  # 3 plots + 1 spacer
    for j in range(3):
        ax = fig.add_subplot(gs[base_row + j])
        ax.plot(x, ppg_ir, label=plot_labels[j], color=colors[j])
        if j == 1:
            ax.text(-0.025, 0.5, plot_labels[j], transform=ax.transAxes, rotation = 90, ha='right', va='center', fontsize=12, color='black')

            line = FancyArrowPatch((-0.02, -1.5), (-0.02, 2.5), transform=ax.transAxes, arrowstyle='-', linewidth=1, color='black')
            fig.patches.append(line)
        ax.legend()

axes = fig.get_axes()
con = ConnectionPatch(
    xyA=(0.5, 1.0), coordsA='axes fraction',  # Top center of ax1
    xyB=(0.5, 0.0), coordsB='axes fraction',  # Bottom center of ax2
    axesA=axes[0], axesB=axes[1],
    color='black', linewidth=2
)

fig.add_artist(con)


plt.subplots_adjust(left=0.05, right= 0.95, top=0.95, bottom=0.05, hspace=0.6)
plt.savefig('V:/test.png')
plt.show()
"""